<a href="https://colab.research.google.com/github/brokentwigs/Colab-Analysis/blob/main/Data_Preprocessing(FINAL).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 3.4 MB/s eta 0:00:00


In [ ]:
import time

# visualizatoin
import matplotlib.pyplot as plt
import seaborn as sns

# data wrangling
import pandas as pd
import numpy as np

# data preprocessing
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import VarianceThreshold
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn.compose import make_column_transformer
import category_encoders as ce

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# description
description = pd.read_csv('WiDS_Datathon_2020_Dictionary.csv')
description_dict = description.set_index('Variable Name').to_dict(orient='index')
# data
df = pd.read_csv('training_v2.csv')

#print_full(description)
df.head()

,encounter_id,patient_id,hospital_id,hospital_death,age,bmi,elective_surgery,ethnicity,gender,height,...,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem
0,66154,25312,118,0,68.0,22.73,0,Caucasian,M,180.3,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Sepsis,Cardiovascular
1,114252,59342,81,0,77.0,27.42,0,Caucasian,F,160.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Respiratory,Respiratory
2,119783,50777,118,0,25.0,31.95,0,Caucasian,F,172.7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Metabolic,Metabolic
3,79267,46918,118,0,81.0,22.64,1,Caucasian,F,165.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Cardiovascular,Cardiovascular
4,92056,34377,33,0,19.0,NaN,0,Caucasian,M,188.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Trauma,Trauma


In [ ]:
# Returns category description

def category(description, string):

    return description[description.Category==string]

# Returns highly correlated features
def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

In [ ]:
# Parameter definitions
test_size = 0.2 # proportion for train versus test+val split
val_size = 0.5 # proportion for test versus val split
random_state = 42  # random state is used to set a seed for randomness, which is only relevant for reproducibility purposes
max_missing = 0.8  # maximum percentage of missing values for a column to be dropped

In [ ]:
X = df.copy().drop(['hospital_death'], axis=1)
y = df['hospital_death'].copy()
y_apache = df['apache_4a_hospital_death_prob'].copy()

""" SPLIT DATA SET """
# split the dataset into train and test+validation set
(
    X_train,
    X_test,
    y_train,
    y_test,
    y_apache_train,
    y_apache_test,
    ) = train_test_split(X, y, y_apache,
                         test_size=test_size, # used for testing and validation
                         random_state=random_state # for reproducibility
                        )
# split the test set into test + validation set
(
    X_val,
    X_test,
    y_val,
    y_test,
    y_apache_val,
    y_apache_test,
    ) = train_test_split(X_test, y_test, y_apache_test,
                         test_size=val_size, # used for testing and validation
                         random_state=random_state # for reproducibility
                        )

In [ ]:
apache_cov = category(description,"APACHE covariate")
vitals = category(description,"vitals")
labs = category(description,"labs")

corr_features_apache_cov = correlation(X_train[apache_cov["Variable Name"]], 0.85)
corr_features_vitals = correlation(X_train[vitals["Variable Name"]], 0.85)
corr_features_labs = correlation(X_train[labs["Variable Name"]], 0.85)

In [ ]:
# DROP THESE COLUMNS
to_drop = set()
to_drop.update(['apache_2_bodysystem', 'apache_4a_icu_death_prob', 'encounter_id', 'patient_id', 'weight', 'height',
                'icu_stay_type', 'readmission_status', 'hospital_id', 'icu_id',
                'apache_4a_hospital_death_prob', 'apache_4a_icu_death_prob'])
to_drop.update(corr_features_apache_cov)
to_drop.update(corr_features_vitals)
to_drop.update(corr_features_labs)

In [ ]:
X_train = X_train.drop(to_drop, axis=1)
X_val = X_val.drop(to_drop, axis=1)
X_test = X_test.drop(to_drop, axis=1)

In [ ]:
"""MISSING VALUES"""
# drop columns with many missing values
missing = X_train.isna().sum() > max_missing * len(X_train)
missing = missing[missing].index
X_train = X_train.drop(missing, axis=1)
X_val = X_val.drop(missing, axis=1)
X_test = X_test.drop(missing, axis=1)

In [ ]:
def customprocessor(X, y):
    num = [col for col in X.columns if X[col].dtypes != 'O' and ~X[col].isin([0.0,1.0,np.nan]).all()]
    cat = [col for col in X.columns if X[col].dtypes == 'O' or X[col].isin([0.0,1.0,np.nan]).all()]

    # Imputing numerical values
    linearreg = LinearRegression()
    imp = IterativeImputer(estimator=linearreg, missing_values=np.nan, max_iter=5, imputation_order='roman', verbose=2, random_state=42)
    X[num] = imp.fit_transform(X[num])

    # Imputing categorical values
    impcat = SimpleImputer(strategy="most_frequent")
    X[cat] = impcat.fit_transform(X[cat])

    # Turn 'gender' column into a binary column
    gender = {'M': 1,'F': 0}
    X.gender = [gender[item] for item in X.gender]

    # Target encoding with additive smoothing
    def calc_smooth_mean(X, y, by, on, m):
        df = pd.concat([X, y], axis=1)

        # Compute the global mean
        mean = df[on].mean()

        # Compute the number of values and the mean of each group
        agg = df.groupby(by)[on].agg(['count', 'mean'])
        counts = agg['count']
        means = agg['mean']

        # Compute the "smoothed" means
        smooth = (counts * means + m * mean) / (counts + m)

        # Replace each value by the according smoothed mean
        return X[by].map(smooth)

    X['ethnicity'] = calc_smooth_mean(X, y, by='ethnicity', on='hospital_death', m=10)
    X['hospital_admit_source'] = calc_smooth_mean(X, y, by='hospital_admit_source', on='hospital_death', m=10)
    X['icu_admit_source'] = calc_smooth_mean(X, y, by='icu_admit_source', on='hospital_death', m=10)
    X['icu_type'] = calc_smooth_mean(X, y, by='icu_type', on='hospital_death', m=10)
    X['apache_3j_bodysystem'] = calc_smooth_mean(X, y, by='apache_3j_bodysystem', on='hospital_death', m=10)

    return X

In [ ]:
# Applying pre-processing steps
X_train = customprocessor(X_train, y_train)
X_val = customprocessor(X_val, y_val)
X_test = customprocessor(X_test, y_test)

[IterativeImputer] Completing matrix with shape (13067, 90)
[IterativeImputer] Ending imputation round 1/5, elapsed time 6.06
[IterativeImputer] Change: 4995.048672367039, scaled tolerance: 8.716669631999999 
[IterativeImputer] Ending imputation round 2/5, elapsed time 12.41
[IterativeImputer] Change: 9826.665597342115, scaled tolerance: 8.716669631999999 
[IterativeImputer] Ending imputation round 3/5, elapsed time 18.73
[IterativeImputer] Change: 1696.9452336419388, scaled tolerance: 8.716669631999999 
[IterativeImputer] Ending imputation round 4/5, elapsed time 24.61
[IterativeImputer] Change: 1032.988215778667, scaled tolerance: 8.716669631999999 
[IterativeImputer] Ending imputation round 5/5, elapsed time 30.79
[IterativeImputer] Change: 658.2145252855908, scaled tolerance: 8.716669631999999 
[IterativeImputer] Completing matrix with shape (1633, 90)
[IterativeImputer] Ending imputation round 1/5, elapsed time 1.35
[IterativeImputer] Change: 7106.153808494147, scaled tolerance: 8

In [ ]:
X_train.shape, X_test.shape, X_val.shape

((13067, 109), (1634, 109), (1633, 109))

In [ ]:
X_test.dtypes

age                            float64
bmi                            float64
elective_surgery                object
ethnicity                      float64
gender                           int64
                                ...   
immunosuppression               object
leukemia                        object
lymphoma                        object
solid_tumor_with_metastasis     object
apache_3j_bodysystem           float64
Length: 109, dtype: object